In [12]:
import pandas as pd

In [13]:
account    = pd.read_csv("dados/cleaned/account.csv")
card       = pd.read_csv("dados/cleaned/card.csv")
client     = pd.read_csv("dados/cleaned/client.csv")
disp       = pd.read_csv("dados/cleaned/disp.csv")
district   = pd.read_csv("dados/cleaned/district.csv")
loan_comp  = pd.read_csv("dados/cleaned/loan_comp.csv")
loan_dev   = pd.read_csv("dados/cleaned/loan_dev.csv")
trans_comp = pd.read_csv("dados/cleaned/trans_comp.csv")
trans_dev  = pd.read_csv("dados/cleaned/trans_dev.csv")

In [14]:
all_merged = []

In [15]:
for i, loan in enumerate([loan_comp, loan_dev]):
    ## Client and disctrict
    client_dist = pd.merge(client, district, left_on="district_id", right_on="id", suffixes=("_client", "_dist"))

    ## Loan and account
    loan_acc = pd.merge(loan, account, on="account_id", how="left", suffixes=("_loan", "_acc"))
    ## And disp
    loan_acc_disp = pd.merge(loan_acc, disp, on="account_id", how="inner")
    
    ## Loan/Account/Disp with Client/District
    df = pd.merge(loan_acc_disp, client_dist, on="client_id", how="inner", suffixes=("_acc", "_client"))
    
    ## Loan/Account/Disp/Client/District with Card
    df = pd.merge(df, card, on="disp_id", how="left")

    ## Loan/Account/Disp/Client/District/Card with transactions
    
    ## If competition (i == 0)
    if not i:
        df = pd.merge(df, trans_comp, on="account_id", how="inner")
    else:
        df = pd.merge(df, trans_dev, on="account_id", how="inner")
    
    df["status"].replace([1, -1], [0, 1], inplace=True)

    all_merged.append(df)

In [16]:
columns_to_drop = [
    "account_id",
    "client_id",
    "district_id_acc",
    "district_id_client",
    ## LOAN
    "loan_date",
    ## DISP
    "disp_id",
    ## ACC
    "acc_creation_date",
    "frequency",
    ## CARD
    "card_issued_year",
    "card_issued_month",
    "card_issued_day",
    ## DIST
    "city",
    "region",
    "id",
    "region_zone"
]


In [17]:
all_merged[0].drop(columns=columns_to_drop, axis = 1, inplace = True)

In [18]:
all_merged[1].drop(columns=columns_to_drop, axis = 1, inplace = True)

In [19]:
all_merged[0].columns

Index(['loan_id', 'amount', 'duration', 'payments', 'status',
       'acc_age_at_loan', 'is_account_shared', 'sex', 'age', 'num_inhab',
       'perc_urban_inhab', 'avg_salary', 'enterp_per_1000', 'num_crimes',
       'unemployment', 'unemployment_delta', 'crimes_delta',
       'type_card_classic', 'type_card_gold', 'type_card_junior',
       'type_card_other', 'has_card', 'min_balance', 'avg_balance',
       'max_balance', 'sanctions', 'only_to_na'],
      dtype='object')

In [20]:
all_merged[0].to_csv("dados/ready/comp.csv", index = False)
all_merged[1].to_csv("dados/ready/dev.csv",  index = False)